In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"for dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))"

In [20]:
# os.listdir('../input/dogs-vs-cats/train/train')

In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [22]:
data_gen = ImageDataGenerator(rotation_range=30, zoom_range=0.2, shear_range=0.2, horizontal_flip=True, rescale=1/255)

In [23]:
filenames = os.listdir('../input/dogs-vs-cats/train/train')
classes = []
for file in filenames:
    classes.append(file.split('.')[0])

In [24]:
len(filenames), len(classes)

(25000, 25000)

In [25]:
df = pd.DataFrame({'FILENAME':filenames, 'CLASS':classes})

In [26]:
df.head()

,FILENAME,CLASS
0,cat.12461.jpg,cat
1,dog.3443.jpg,dog
2,dog.7971.jpg,dog
3,dog.10728.jpg,dog
4,dog.1942.jpg,dog


In [27]:
train_data = data_gen.flow_from_dataframe(df, '../input/dogs-vs-cats/train/train/',x_col='FILENAME',y_col='CLASS',target_size=(224,224))

Found 25000 validated image filenames belonging to 2 classes.


In [ ]:
train_data.class_indices

In [ ]:
# from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
# from tensorflow.keras.models import Sequential

In [ ]:
# model = Sequential()
# model.add(Conv2D(32,(3,3), input_shape=(224,224,3)))
# model.add(MaxPool2D())
# model.add(Conv2D(64,(3,3)))
# model.add(MaxPool2D())
# model.add(Conv2D(64,(5,5)))
# model.add(MaxPool2D())
# model.add(Flatten())
# model.add(Dense(2, activation = 'softmax'))

In [10]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Model

In [5]:
vgg16 = VGG16( = False, input_shape=(224,224,3))

58892288/58889256 [==============================] - 1s 0us/step


In [6]:
for layers in vgg16.layers:
    layers.trainable = False

In [12]:
vgg16.summary()
flat_layer = Flatten()(vgg16.output)
final_layer = Dense(2,activation='softmax')(flat_layer)

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [15]:
model = Model(inputs=vgg16.input,outputs=final_layer)

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [17]:
model.compile(loss='categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(train_data, epochs=10,batch_size=24)

Epoch 1/10
253/782 [========>.....................] - ETA: 4:18 - loss: 0.7980 - accuracy: 0.7710

In [ ]:
len(train_data)